In [ ]:
from helper import *
import sys

In [ ]:
with open("./credentials.json") as r:
    config = json.loads(r.read())
minio_config = dict(
    key=config["accessKey"],
    secret=config["secretKey"],
    client_kwargs={
        'endpoint_url': config["url"]
    }
)

with open("./api_config.json") as r:
    config = json.loads(r.read())
api_config = dict(
    url=config["url"],
    token=config["token"],
)

In [ ]:
url = api_config["url"]
token = api_config["token"]
headers = {"Authorization": "Bearer " + token}
topic = "service_ipd_drug"
api_config["headers"] = headers
api_config["topic"] = topic

try:
  r = get_process(**api_config)
except Exception as ex:
  print(ex)
  sys.exit(0)

## set config to processing 
hospcode = r["hospcode"]
process_id = r["process_id"]
api_config["process_id"] = process_id
source_path = "s3://mophgw/{}".format(r["path"])
destination_path = "s3://phr/hisgw/{}/{}/{}".format(hospcode, topic,process_id)
local_code_path = "s3://mapcode/{}/localcode".format(hospcode)
std_code_path = "s3://mapcode/{}/stdcode".format(hospcode)

In [ ]:
hospital = pd.read_csv(f"{std_code_path}/", engine="c",
                       dtype=pd.StringDtype(), storage_options=minio_config)

In [ ]:
person = read_csv(minio_config, path=source_path, filename="person.csv.gz")

In [ ]:
person_merge_local = dict(
    title=["title_code", "code"],
    gender=["gender", "code"],
    rh_blood_group=["rh_blood_group", "code"],
    blood_group=["blood_group", "code"],
    nationality=["nationality_code", "code"],
    marital_status=["marital_status", "code"],
)
person = merge_codes(minio_config, person, local_code_path,
                     prefix_type="local", merge_config=person_merge_local)

In [ ]:
person_merge_std = dict(
    titles=["title_localcode", "localcode"],
    genders=["gender_localcode", "localcode"],
    rh_blood_groups=["rh_blood_group_localcode", "localcode"],
    blood_groups=["blood_group_localcode", "localcode"],
    nationalities=["nationality_localcode", "localcode"],
    marital_statuses=["marital_status_localcode", "localcode"],
)
person = merge_codes(minio_config, person, std_code_path,
                     prefix_type="std", merge_config=person_merge_std)

In [ ]:
cols_rename = dict(
    group="groups",
    nationality="nationalities",
    status="statuses",
    gender="genders",
    title="titles"
)
person = rename_columns(person, cols_rename)

In [ ]:
service_ipd = read_csv(
    minio_config, path=source_path, filename="service_ipd.csv.gz")

In [ ]:
service_ipd_drug = read_csv(minio_config, path=source_path, filename="service_ipd_drug.csv.gz")
